In [ ]:
import JSSP
import gym
import time
import numpy as np
import random

# Environment Initialization

In [ ]:
def create_env(instance_path):
    env_name = "JSSP-v0"
    env = gym.make(env_name, instance_path = instance_path)
    print("Environment Created for: ", instance_path)
    print("Observation space: \n", env.observation_space)
    print("Action space: \n", env.action_space)
    return env

In [ ]:
env1 = create_env("instance1.txt")
env3 = create_env("instance3.txt")
env4 = create_env("instance4.txt")

# Baselines

## 1. Random Sampling

In [ ]:
def random_sampling(env, episodes):
    env.reset()
    max_score = -100000
    max_episode = -1
    max_action_list = []
    max_time_list = []
    for episode in range(1, episodes+1):
        env.reset()
        done = False
        score = 0
        action_list = []
        time_list = []
        while not done:
            #env.render()
            action = env.action_space.sample()
            if action != env.action_space.n -1:
                action_list.append(env.legal_allocation_list[action])
                time_list.append(env.time)
                print('Episode:{} Allocation:{} Time:{}'.format(episode, env.legal_allocation_list[action], env.time))
            n_state, reward, done, info = env.step(action)
            score+=reward
        print('Episode:{} Total_reward:{}'.format(episode, score))
        if score >= max_score:
            max_score = score
            max_episode = episode
            max_action_list = action_list
            max_time_list = time_list
    print('From {}th Episode best policy has reward {}'.format(max_episode, max_score))
    for i in range(len(max_action_list)):
        print('The allocation chose at time {} is {}'.format(max_time_list[i], max_action_list[i]))

random_sampling(env4, 100)

## 2. Q-Learning

In [ ]:
def update(Q, s_key, s_next_key, action, r, eta, gma):

    if s_key not in Q.keys():
        Q[s_key] = {}

    if s_next_key not in Q.keys() or Q[s_next_key] == {}:
        max_next_s_Q = 0
    else:
        max_next_s_Q = max(Q[s_next_key].values())

    if action not in Q[s_key].keys():
        Q[s_key][action] = 0
    Q[s_key][action] = Q[s_key][action] + eta * (r + gma * max_next_s_Q - Q[s_key][action])

    return Q

In [ ]:
def q_learning(env, epis):
    # 1. Load Environment and Q-table structure
    Q = {}
    # 2. Parameters of Q-learning
    eta = .628
    gamma = .9
    epsilon = .1
    rev_list = [] # rewards per episode calculate
    # 3. Q-learning Algorithm
    for i in range(epis):
        # Reset environment
        s = env.reset()
        rAll = 0
        d = False
        # The Q-Table learning algorithm
        while not d:
            # env.render()
            # Choose action from Q table
            s_key = tuple(np.concatenate((s[1], s[0])))
            if s_key not in Q.keys() or Q[s_key] == {} or random.uniform(0, 1) < epsilon:
                a = env.action_space.sample()
            else:
                a = max(Q[s_key], key = Q[s_key].get)
            #Get new state & reward from environment
            s_next, r, d, _ = env.step(a)
            #Update Q-Table with new knowledge
            s_next_key = tuple(np.concatenate((s_next[1], s_next[0])))
            Q = update(Q, s_key, s_next_key, a, r, eta, gamma)
            rAll += r
            s = s_next
        rev_list.append(rAll)
        print("Episode: " + str(i) + " has reward " + str(rAll))
        # env.render()
    # Code will stop at d == True, and render one state before it
    print("Reward Sum on all episodes " + str(sum(rev_list)/epis))
    print("Final Values Q-Table")
    print(Q)
    return rev_list

In [ ]:
rev_list = q_learning(env3, 500)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(rev_list)
plt.show

In [ ]:
plt.plot(rev_list)
plt.show

In [ ]:
random_sampling(env3, 5000)